In [1]:
#Function to pretty print the directory structure
import os
def list_files(startpath):
  for root, dirs, files in os.walk(startpath):
    level = root.replace(startpath, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print('{}{}/'.format(indent, os.path.basename(root)))
    subindent = ' ' * 4 * (level + 1)

list_files('/Users/kgupta/Desktop/dat')

dat/
    test/
        One/
        Ten/
        Two/
        Five/
    train/
        One/
        Ten/
        Two/
        Five/


In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

#Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#CNN Architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten, GlobalAvgPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [3]:
#Data Augmentation and Pre-Processing

train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   rotation_range=90,  
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#Training Set
training_set = train_datagen.flow_from_directory('/Users/kgupta/Desktop/dat/train', 
                                                 target_size = (200,200), 
                                                 batch_size = 32, 
                                                 class_mode='categorical')

#Test Set
test_set = test_datagen.flow_from_directory('/Users/kgupta/Desktop/dat/test', 
                                            target_size = (200,200), 
                                            batch_size = 32, 
                                            class_mode = 'categorical')

Found 6755 images belonging to 4 classes.
Found 1497 images belonging to 4 classes.


In [4]:
#CNN Architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten, GlobalAvgPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
model = Sequential()

# CNN network
model.add( Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(200,200, 3)) )
model.add( MaxPool2D(2,2) )

model.add( Conv2D(32, (3,3), activation='relu', padding='same') )
model.add( MaxPool2D(2,2) )

model.add( Conv2D(64, (3,3), activation='relu', padding='same') )
model.add( MaxPool2D(2,2) )
# Transition between CNN and MLP
model.add(Flatten())

# MLP network
model.add( Dense(512, activation='relu') )

model.add( Dense(4, activation='softmax') )

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 40000)             0

2022-01-11 16:03:28.398796: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-11 16:03:28.400936: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#Setting up Training Scheme
optm = Adam(learning_rate = 1e-3) #Loss fuction

model.compile(optimizer = optm, loss = 'categorical_crossentropy', metrics = ['acc']) 

#Setting Up callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

callbacks = [
    ReduceLROnPlateau(patience=5, factor=0.1, verbose=True),
    ModelCheckpoint('best_model', save_best_only=True),
]

In [ ]:
#Training 
history = model.fit(training_set, 
                    epochs=200, 
                    validation_data = test_set, 
                    batch_size=64, 
                    steps_per_epoch = 120,
                    callbacks = callbacks);

2022-01-11 16:03:45.762060: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/200
120/120 [==============================] - 68s 563ms/step - loss: 1.8401 - acc: 0.3446 - val_loss: 0.9438 - val_acc: 0.5184


2022-01-11 16:04:53.945888: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: best_model/assets
Epoch 2/200
120/120 [==============================] - 78s 648ms/step - loss: 0.8963 - acc: 0.5903 - val_loss: 0.8439 - val_acc: 0.5845
INFO:tensorflow:Assets written to: best_model/assets
Epoch 3/200
120/120 [==============================] - 82s 680ms/step - loss: 0.8189 - acc: 0.6314 - val_loss: 0.8291 - val_acc: 0.6313
INFO:tensorflow:Assets written to: best_model/assets
Epoch 4/200
120/120 [==============================] - 79s 656ms/step - loss: 0.7164 - acc: 0.6806 - val_loss: 0.6618 - val_acc: 0.7168
INFO:tensorflow:Assets written to: best_model/assets
Epoch 5/200
120/120 [==============================] - 82s 685ms/step - loss: 0.6324 - acc: 0.7181 - val_loss: 0.6995 - val_acc: 0.6941
Epoch 6/200
120/120 [==============================] - 81s 670ms/step - loss: 0.6457 - acc: 0.7044 - val_loss: 0.7863 - val_acc: 0.6573
Epoch 7/200
120/120 [==============================] - 88s 732ms/step - loss: 0.5970 - acc: 0.7293 - val_los

120/120 [==============================] - 82s 678ms/step - loss: 0.1012 - acc: 0.9610 - val_loss: 0.3022 - val_acc: 0.8945
Epoch 54/200
120/120 [==============================] - 81s 675ms/step - loss: 0.1040 - acc: 0.9639 - val_loss: 0.3002 - val_acc: 0.8958
Epoch 55/200
120/120 [==============================] - 81s 676ms/step - loss: 0.1118 - acc: 0.9570 - val_loss: 0.3000 - val_acc: 0.8958

Epoch 00055: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
Epoch 56/200
120/120 [==============================] - 82s 677ms/step - loss: 0.0999 - acc: 0.9586 - val_loss: 0.3000 - val_acc: 0.8958
Epoch 57/200
120/120 [==============================] - 81s 672ms/step - loss: 0.0998 - acc: 0.9643 - val_loss: 0.3001 - val_acc: 0.8958
Epoch 58/200
120/120 [==============================] - 81s 671ms/step - loss: 0.1121 - acc: 0.9536 - val_loss: 0.3001 - val_acc: 0.8958
Epoch 59/200
120/120 [==============================] - 81s 672ms/step - loss: 0.1077 - acc: 0.9574 - val_los

120/120 [==============================] - 82s 685ms/step - loss: 0.1024 - acc: 0.9611 - val_loss: 0.2999 - val_acc: 0.8951
Epoch 107/200
120/120 [==============================] - 81s 677ms/step - loss: 0.1015 - acc: 0.9599 - val_loss: 0.2999 - val_acc: 0.8951
Epoch 108/200
120/120 [==============================] - 82s 683ms/step - loss: 0.1009 - acc: 0.9639 - val_loss: 0.2999 - val_acc: 0.8951
Epoch 109/200
120/120 [==============================] - 82s 681ms/step - loss: 0.0894 - acc: 0.9679 - val_loss: 0.2999 - val_acc: 0.8951
Epoch 110/200
120/120 [==============================] - 82s 679ms/step - loss: 0.1010 - acc: 0.9628 - val_loss: 0.2999 - val_acc: 0.8951

Epoch 00110: ReduceLROnPlateau reducing learning rate to 1.0000000664932204e-18.
Epoch 111/200
120/120 [==============================] - 82s 681ms/step - loss: 0.0972 - acc: 0.9636 - val_loss: 0.2999 - val_acc: 0.8951
Epoch 112/200
120/120 [==============================] - 82s 681ms/step - loss: 0.1054 - acc: 0.9624 - v

Epoch 160/200
120/120 [==============================] - 80s 668ms/step - loss: 0.1034 - acc: 0.9609 - val_loss: 0.2999 - val_acc: 0.8951

Epoch 00160: ReduceLROnPlateau reducing learning rate to 1.0000001235416984e-28.
Epoch 161/200
 85/120 [====================>.........] - ETA: 20s - loss: 0.0934 - acc: 0.9653

In [ ]:
df_history = pd.DataFrame(history.history)

max_val_acc = df_history['val_acc'].max()
index = df_history[df_history['val_acc'] == max_val_acc].index
_, train_loss, train_acc, val_loss, _, _ = [k for k in df_history.iloc[index[0]]]

# Printing the Best Results
print('Best Validation Scores:\n')
print('Validation_Accuracy : {}'.format(max_val_acc))
print('Validation_Loss : {}'.format(val_loss))
print('Training_Accuracy : {}'.format(train_acc))
print('Training_Loss : {}'.format(train_loss))

In [ ]:
#Loading the model
from tensorflow.keras.models import load_model
model = load_model('/Users/kgupta/Desktop/best_model/')

#Taking Some Test Images
test = {'Images' : [],
        'Input' : []}
from PIL import Image
import os
dir_path = '/Users/kgupta/Desktop/r/'

for file in os.listdir(dir_path):
  #Load Image
  im = Image.open(dir_path + file)

  #Resize 
  im = im.resize((400,300))

  #Store Sized Image File
  test['Images'].append(im)

  #Rescale and Store normalized array
  im = Image.open(dir_path + file)
  im = im.resize((200,200))
  im = np.asarray(im)
  im = im/255
  im = np.expand_dims(im,axis=0)
  test['Input'].append(im)

In [ ]:
# Plotting Images
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,7),dpi=100)
rows, columns = 3,3

for i in range(len(test['Images'])):
  fig.add_subplot(rows, columns, i+1)
  plt.axis('off')
  plt.title('Image '+str(i + 1))
  plt.imshow(test['Images'][i])

In [ ]:
Class_labels = os.listdir('/Users/kgupta/Desktop/dat/train')

In [ ]:
#Getting Prediction
prediction_labels = []
for inp_img in test['Input']:
  pred_prob = model.predict(inp_img)
  prediction = Class_labels[np.argmax(pred_prob)]
  prediction_labels.append(prediction)

#Plotting Results
fig = plt.figure(figsize=(10,9),dpi=100)
rows, columns = 3,3

for i,pred in zip(range(6),prediction_labels):
  fig.add_subplot(rows, columns, i+1)
  plt.axis('off')
  plt.title('Image '+str(i + 1)+'\n Prediction : {} Rupee'.format(pred))
  plt.imshow(test['Images'][i])